# Perform Experiments with DeepFace on My dataset

In [3]:
# built-in dependencies
import os

# 3rd party dependencies
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.datasets import fetch_lfw_pairs
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from deepface import DeepFace
import random

In [4]:
print(f"This experiment is done with pip package of deepface with {DeepFace.__version__} version")

This experiment is done with pip package of deepface with 0.0.91 version


### Configuration Sets

In [5]:
# all configuration alternatives for 4 dimensions of arguments
alignment = [False, True]
models = ["Facenet512", "ArcFace"]
detectors = ["opencv"]
metrics = ["euclidean", "cosine"]
expand_percentage = 0

### Create Required Folders if necessary

In [6]:
target_paths = ["lfwe", "dataset", "outputs", "outputs/test", "results"]
for target_path in target_paths:
    if os.path.exists(target_path) != True:
        os.mkdir(target_path)
        print(f"{target_path} is just created")

### Load Dataset

In [7]:
pairs_touch = "outputs/test_lfwe.txt"
instances = 1000 #pairs.shape[0]

### Choice 1 Mimic to fetch_lfw_pairs (Not use because data are imbalance)

In [ ]:
dataset_path = "../data_db"
target_path = "dataset/test_lfw.npy"
labels_path = "dataset/test_labels.npy"

def create_random_pairs(data_path, num_pairs=1000):
    images = []
    labels = []
    
    class_folders = sorted(os.listdir(data_path))
    
    for idx, class_folder in enumerate(class_folders):
        class_path = os.path.join(data_path, class_folder)
        image_files = sorted(os.listdir(class_path))
        
        for img_file in image_files:
            img_path = os.path.join(class_path, img_file)
            img = cv2.imread(img_path)
            if img is None or img.size == 0:
                print(f"Warning: Failed to load image: {img_path}")
            else:
                images.append(img)
                labels.append(idx)
    
    # Create all possible pairs
    pairs = []
    pair_labels = []
    
    for i in range(len(images)):
        for j in range(i+1, len(images)):
            pairs.append([images[i], images[j]])
            pair_labels.append(1 if labels[i] == labels[j] else 0)
    
    # Randomly select 1000 pairs
    if len(pairs) >= num_pairs:
        selected_indices = random.sample(range(len(pairs)), num_pairs)
        random_pairs = [pairs[i] for i in selected_indices]
        random_labels = [pair_labels[i] for i in selected_indices]
    else:
        print(f"Warning: Not enough pairs generated ({len(pairs)}). Check your dataset.")
        random_pairs = pairs
        random_labels = pair_labels
    
    return np.array(random_pairs), np.array(random_labels)

if not os.path.exists(target_path) or not os.path.exists(labels_path):
    pairs, labels = create_random_pairs(dataset_path, num_pairs=1000)
    np.save(target_path, pairs)
    np.save(labels_path, labels)
else:
    pairs = np.load(target_path)
    labels = np.load(labels_path)

print(f"Total pairs: {len(pairs)}")
print(f"Positive pairs: {np.sum(labels == 1)}")
print(f"Negative pairs: {np.sum(labels == 0)}")

In [27]:
# Simulate test if i use fetch_lfw_pairs (Not use)
target_path = "dataset/test_lfw.npy"
labels_path = "dataset/test_labels.npy"

if os.path.exists(target_path) != True:
    fetch_lfw_pairs = fetch_lfw_pairs(subset = 'test', data_home='../data_db'
                                  , color = True
                                  , resize = 2
                                  , funneled = False
                                  , slice_=None
                                 )
    pairs = fetch_lfw_pairs.pairs
    labels = fetch_lfw_pairs.target
    target_names = fetch_lfw_pairs.target_names
    np.save(target_path, pairs)
    np.save(labels_path, labels)
else:
    if os.path.exists(pairs_touch) != True:
        # loading pairs takes some time. but if we extract these pairs as image, no need to load it anymore
        pairs = np.load(target_path)
    labels = np.load(labels_path)    

### Choice 2 Balance the number of positive and negative pairs

In [9]:
# Define paths
dataset_path = "../data_db"
target_path = "dataset/test_lfw.npy"
labels_path = "dataset/test_labels.npy"

# Desired image size and number of channels
image_size = (224, 224)  # Adjust this size based on your needs
num_channels = 3  # 3 for RGB, 1 for grayscale

# Function to load images from the dataset folder and resize them
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, image_size)  # Resize image
            if num_channels == 3 and len(img.shape) == 2:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # Convert grayscale to RGB
            elif num_channels == 1 and len(img.shape) == 3:
                img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # Convert RGB to grayscale
            images.append(img)
    return images

# Initialize empty lists for pairs and labels
positive_pairs = []
negative_pairs = []

# Load all images and store them in a dictionary by person name
all_images = {}
for person_name in os.listdir(dataset_path):
    person_folder = os.path.join(dataset_path, person_name)
    if os.path.isdir(person_folder):
        images = load_images_from_folder(person_folder)
        if len(images) > 1:  # Only consider folders with more than 1 image
            all_images[person_name] = images

# Create positive pairs (same person)
for person_name, images in all_images.items():
    num_images = len(images)
    for i in range(num_images):
        for j in range(i + 1, num_images):
            positive_pairs.append([images[i], images[j]])

# Create negative pairs (different persons)
person_names = list(all_images.keys())
for i in range(len(person_names)):
    for j in range(i + 1, len(person_names)):
        if len(positive_pairs) > len(negative_pairs):
            person_name_1 = person_names[i]
            person_name_2 = person_names[j]
            img1 = random.choice(all_images[person_name_1])  # Randomly select 1 image from person i
            img2 = random.choice(all_images[person_name_2])  # Randomly select 1 image from person j
            negative_pairs.append([img1, img2])

# Balance the number of positive and negative pairs
num_pairs_to_use = min(len(positive_pairs), len(negative_pairs))
pairs = positive_pairs[:num_pairs_to_use] + negative_pairs[:num_pairs_to_use]
labels = [1] * num_pairs_to_use + [0] * num_pairs_to_use

# Shuffle the pairs and labels
combined = list(zip(pairs, labels))
random.shuffle(combined)
pairs, labels = zip(*combined)

# Convert lists to numpy arrays
pairs = np.array(pairs)
labels = np.array(labels)

# Save the pairs and labels arrays
np.save(target_path, pairs)
np.save(labels_path, labels)

print(f"Total pairs: {len(pairs)}")
print(f"Positive pairs: {np.sum(labels == 1)}")
print(f"Negative pairs: {np.sum(labels == 0)}")

Total pairs: 2162
Positive pairs: 1081
Negative pairs: 1081


### Save image pairs into file system

In [11]:
for i in tqdm(range(0, instances)):
    img1_target = f"lfwe/test/{i}_1.jpg"
    img2_target = f"lfwe/test/{i}_2.jpg"
    #print(f"Pair {i}: img1 size = {pairs[i][0].size}, img2 size = {pairs[i][1].size}")

    if os.path.exists(img1_target) != True:
        img1 = pairs[i][0]
        # plt.imsave(img1_target, img1/255) #works for my mac
        img1 = cv2.cvtColor(pairs[i][0], cv2.COLOR_BGR2RGB)
        plt.imsave(img1_target, img1) #works for my debian
    
    if os.path.exists(img2_target) != True:
        img2 = pairs[i][1]
        # plt.imsave(img2_target, img2/255) #works for my mac
        img2 = cv2.cvtColor(pairs[i][1], cv2.COLOR_BGR2RGB)
        plt.imsave(img2_target, img2) #works for my debian
    
if os.path.exists(pairs_touch) != True:
    open(pairs_touch,'a').close()

100%|██████████| 1000/1000 [00:00<00:00, 38560.87it/s]


### Perform Experiments

This block will save the experiments results in outputs folder

In [12]:
# Define how many instances you want to process

for model_name in models:
    for detector_backend in detectors:
        for distance_metric in metrics:
            for align in alignment:
                
                if detector_backend == "skip" and align is True:
                    # Alignment is not possible for a skipped detector configuration
                    continue
                
                alignment_text = "aligned" if align is True else "unaligned"
                task = f"{model_name}_{detector_backend}_{distance_metric}_{alignment_text}"
                output_file = f"outputs/test/{task}.csv"
                if os.path.exists(output_file) is True:
                     #print(f"{output_file} is available already")
                     continue
                
                distances = []
                for i in tqdm(range(0, instances), desc = task):
                    img1_target = f"lfwe/test/{i}_1.jpg"
                    img2_target = f"lfwe/test/{i}_2.jpg"
                    result = DeepFace.verify(
                        img1_path=img1_target,
                        img2_path=img2_target,
                        model_name=model_name,
                        detector_backend=detector_backend,
                        distance_metric=distance_metric,
                        align=align,
                        enforce_detection=False,
                        expand_percentage=expand_percentage,
                    )
                    distance = result["distance"]
                    distances.append(distance)
                # -----------------------------------
                df = pd.DataFrame({"actuals": labels[:instances]})
                df["distances"] = distances
                df.to_csv(output_file, index=False)

ArcFace_opencv_cosine_aligned: 100%|██████████| 1000/1000 [07:38<00:00,  2.18it/s]


### Calculate Results

Experiments were responsible for calculating distances. We will calculate the best accuracy scores in this block.

In [13]:
data = [[0 for _ in range(len(models))] for _ in range(len(detectors))]
base_df = pd.DataFrame(data, columns=models, index=detectors)

In [14]:
for is_aligned in alignment:
    for distance_metric in metrics:

        current_df = base_df.copy()
        
        target_file = f"results/pivot_{distance_metric}_with_alignment_{is_aligned}.csv"
        if os.path.exists(target_file):
            continue
        
        for model_name in models:
            for detector_backend in detectors:

                align = "aligned" if is_aligned is True else "unaligned"

                if detector_backend == "skip" and is_aligned is True:
                    # Alignment is not possible for a skipped detector configuration
                    align = "unaligned"

                source_file = f"outputs/test/{model_name}_{detector_backend}_{distance_metric}_{align}.csv"
                df = pd.read_csv(source_file)
                  
                positive_mean = df[(df["actuals"] == True) | (df["actuals"] == 1)]["distances"].mean()
                negative_mean = df[(df["actuals"] == False) | (df["actuals"] == 0)]["distances"].mean()

                distances = sorted(df["distances"].values.tolist())
                #print(positive_mean)
                #print(negative_mean)
                #print(distances)
                items = []
                precision = []
                recall = []
                f1_scores = []
                for i, distance in enumerate(distances):
                    if distance >= positive_mean and distance <= negative_mean:
                        sandbox_df = df.copy()
                        sandbox_df["predictions"] = False
                        idx = sandbox_df[sandbox_df["distances"] < distance].index
                        sandbox_df.loc[idx, "predictions"] = True

                        actuals = sandbox_df.actuals.values.tolist()
                        predictions = sandbox_df.predictions.values.tolist()
                        accuracy = 100*accuracy_score(actuals, predictions)
                        items.append((distance, accuracy))

                        precision.append(precision_score(actuals, predictions, average="weighted"))
                        recall.append(recall_score(actuals, predictions, average="weighted"))
                        #f1_scores.append(f1_score(actuals, predictions, average="macro"))

                pivot_df = pd.DataFrame(items, columns = ["distance", "accuracy"])
                pivot_df = pivot_df.sort_values(by = ["accuracy"], ascending = False)
                print(model_name)
                print(np.average(precision))
                print(np.average(recall))
                print(f1_scores)
                # encode the labels
                #le = LabelEncoder()
                #le.fit_transform(labels)
                print(classification_report(actuals, predictions, digits=3))
                threshold = pivot_df.iloc[0]["distance"]
                print(f"threshold for {model_name}/{detector_backend} is {threshold}")
                accuracy = pivot_df.iloc[0]["accuracy"]

                print(source_file, round(accuracy, 1))
                current_df.at[detector_backend, model_name] = round(accuracy, 1)
        
        current_df.to_csv(target_file)
        print(f"{target_file} saved")

Facenet512
0.811802453685039
0.7869587628865978
[]
              precision    recall  f1-score   support

           0      0.905     0.506     0.649       506
           1      0.651     0.945     0.771       494

    accuracy                          0.723      1000
   macro avg      0.778     0.726     0.710      1000
weighted avg      0.779     0.723     0.709      1000

threshold for Facenet512/opencv is 22.90168547542397
outputs/test/Facenet512_opencv_euclidean_unaligned.csv 83.1
ArcFace
0.7689264991728961
0.7538173076923077
[]
              precision    recall  f1-score   support

           0      0.753     0.447     0.561       506
           1      0.600     0.850     0.704       494

    accuracy                          0.646      1000
   macro avg      0.677     0.648     0.632      1000
weighted avg      0.678     0.646     0.631      1000

threshold for ArcFace/opencv is 4.886483578563145
outputs/test/ArcFace_opencv_euclidean_unaligned.csv 80.6
results/pivot_euclidean_wi